In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
import time
import os
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import plotly.express as px
import requests
from bs4 import BeautifulSoup

In [ ]:
#находим самый оптимальный вариант для аренды помещения

In [13]:
def get_warehouse_cian(url):
    download_dir = os.path.expanduser("~/Downloads")
    driver = webdriver.Chrome()
    driver.get(url)
    time.sleep(5)
    try:
        button = driver.find_element(By.CSS_SELECTOR, "button[data-name='Button']")
        button.click()
        time.sleep(10)
        file_name = "offers.xlsx"

        # Полный путь к файлу
        file_path = os.path.join(download_dir, file_name)
        df_table = pd.read_excel(file_path, engine='openpyxl')
        driver.quit()
        return df_table
    except Exception as e:
        driver.quit()
        print(type(e))
        print(e)
df_table = get_warehouse_cian('https://www.cian.ru/snyat-sklad/')

<class 'zipfile.BadZipFile'>
File is not a zip file


In [21]:
#привожу таблицу к рабочему состоянию
df_table['Цена'] = df_table['Цена'].str.extract(r'(\d+)', expand=False)
df_table['Площадь'] = df_table['Площадь'].str.extract(r'(\d+)', expand=False)
df_table['Цена'] = df_table['Цена'].str.replace(' ', '').astype(float)
df_table['Площадь'] = df_table['Площадь'].str.replace(' ', '').astype(float)
df_table['Площадь/Цена'] = df_table['Площадь'] / df_table['Цена']
df_sorted_1 = df_table.sort_values(by='Площадь/Цена', ascending=False)
df_sorted_1['ID  объявления'] = df_sorted_1['ID  объявления'].astype(str)

In [28]:
top_50 = df_sorted_1.head(50)
fig = px.bar(top_50, x='ID  объявления', y='Площадь/Цена',
             labels={'Площадь/Цена': 'Соотношение площади к цене', 'ID  объявления': 'ID предложения'},
             title='Соотношение площади к цене для каждого предложения',
             text='Площадь/Цена')  


fig.update_layout(height=500, width=1200,
                  xaxis_tickangle=-45,
                  xaxis_title='ID предложения',
                  yaxis_title='Соотношение площади к цене',
                  )


fig.show()

In [29]:
#теперь проверяем с яндекс недвижимостью

In [31]:
def get_warehouse_yandex(driver):
    offers = driver.find_elements(By.CSS_SELECTOR, "div.OffersSerpItem")
    data = []
    offers = WebDriverWait(driver, 20).until(
        EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.OffersSerpItem__info"))
    )
    for offer in offers:
        try:
            try:
                id_offer = offer.find_element(By.CSS_SELECTOR, "a.OffersSerpItem__link").get_attribute("href")
            except NoSuchElementException:
                id_offer = "N/A"
            offer_type_div = offer.find_element(By.CSS_SELECTOR, "div.OffersSerpItem__generalInfoInnerContainer")
            offer_type = offer_type_div.find_element(By.CSS_SELECTOR, "span").text.strip()
            area_div = offer.find_element(By.CSS_SELECTOR, "div.OffersSerpItem__generalInfoInnerContainer")
            area = area_div.find_element(By.CSS_SELECTOR, "span").text.strip()
            price_div = offer.find_element(By.CSS_SELECTOR, "div.Price")
            price = price_div.find_element(By.CSS_SELECTOR, "span").text.strip()
            link = offer.find_element(By.CSS_SELECTOR, "a.OffersSerpItem__link").get_attribute("href")
            data.append({
                "Id": id_offer,
                "Тип": offer_type,
                "Площадь": area,
                "Цена": price,
                "Ссылка": link
            })
        except Exception as e:
            print(f"Ошибка при парсинге карточки: {e}")
    return data

In [32]:
all_data = []
url = "https://realty.yandex.ru/moskva/snyat/kommercheskaya-nedvizhimost/?areaMax=300&areaMin=50&commercialType=WAREHOUSE&commercialType=FREE_PURPOSE"
try:
    while True:
        driver = webdriver.Chrome()
        driver.get(url)
        time.sleep(5) #более простой метод ожидания подсказал чат
        page_data = get_warehouse_yandex(driver)
        all_data.extend(page_data)
        buttons = driver.find_elements(By.CSS_SELECTOR, "button.Button")
        for button in buttons:
            if "Следующая" in button.text:
                next_button = button
                break
        
        if next_button:
            next_button.click()
            
        else:
            print("Кнопка 'Следующая' не найдена. Завершение парсинга.")
            driver.quit()
            break
        url = driver.current_url
        driver.quit()
except Exception as e:
    print(f"Ошибка: {e}")
finally:
    df = pd.DataFrame(all_data)

Ошибка: HTTPConnectionPool(host='localhost', port=61580): Max retries exceeded with url: /session/9ef2b1061b32d983fa61491eed7e3e5b/element/f.93CA346DDB3FD1D1D7CC61C9D8395793.d.116A5DB8177AA75E93BEA57FD0284CC8.e.30/click (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x30af4bf10>: Failed to establish a new connection: [Errno 61] Connection refused'))


In [34]:
df_to_work = df.copy()
df_to_work['Id'] = df['Id'].str.extract(r'(\d+)')
df_to_work['Площадь'] = df['Площадь'].str.extract(r'(\d+)')
df_to_work['Тип'] = df['Тип'].str.extract(r'·\s*(.*?)(?:\s*·|$)')
df_to_work['Цена'] = df_to_work['Цена'].str.replace(' ', '').astype(float)
df_to_work['Площадь'] = df_to_work['Площадь'].str.replace(' ', '').astype(float)
df_to_work['Площадь/Цена'] = df_to_work['Площадь'] / df_to_work['Цена']

In [35]:
df_sorted_2 = df_to_work.sort_values(by='Площадь/Цена', ascending=False)


top_50 = df_sorted_2.head(50)
fig = px.bar(top_50, x='Id', y='Площадь/Цена',
             labels={'Площадь/Цена': 'Соотношение площади к цене', 'Id': 'ID предложения'},
             title='Соотношение площади к цене для каждого предложения',
             text='Площадь/Цена')  


fig.update_layout(height=500, width=1200,
                  xaxis_tickangle=-45,
                  xaxis_title='ID предложения',
                  yaxis_title='Соотношение площади к цене',
                  )


fig.show()

In [ ]:
#видим примерное равенство соотношений, за исключением одного аномального показателя
#среди самых выгодных выбираю предложение с отоплением и проветриванием для растений

In [ ]:
#чтобы понять, какие растения скорее всего захотят взять, возьму информацию с самых популярных предложений маркетплейсов

In [42]:
#делаю список со всеми возможными растениями
def get_plants_names_soup(url):
    html = requests.get(url).content.decode('utf-8')
    soup = BeautifulSoup(html, 'html.parser')
    plant_names = []
    names = soup.find_all('a', {'class' : 'em14'})
    for name in names:
        name = name.text
        plant_names.append(name)
    return plant_names
k=0
all_plant_names = []
while k <= 27:
    url = f"https://www.greeninfo.ru/alphabet.html/letter/{k}"
    k_plant = get_plants_names_soup(url)
    all_plant_names.extend(k_plant)
    k=k+1

In [41]:
def get_popular_seeds_wildberries(url):
    data = []
    driver = webdriver.Chrome()
    driver.get(url)
    wait = WebDriverWait(driver, 10)
    products = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "article")))
    for product in products:
        try:
            title = product.find_element(By.CSS_SELECTOR, "span.product-card__name").text
            link = product.find_element(By.CSS_SELECTOR, "a").get_attribute("href")
            price = product.find_element(By.CSS_SELECTOR, "ins").text
            
            
            data.append({
                "Название": title,
                "Ссылка": link,
                "Цена": price
            })
        except Exception as e:
            print(e)
    driver.quit()
    return data 

final_data = []
i=1
while i <= 20:
    url = f"https://www.wildberries.ru/catalog/0/search.aspx?page={i}&sort=popular&search=%D0%BA%D0%BE%D0%BC%D0%BD%D0%B0%D1%82%D0%BD%D1%8B%D0%B5+%D1%80%D0%B0%D1%81%D1%82%D0%B5%D0%BD%D0%B8%D1%8F"
    i_data = get_popular_seeds_wildberries(url)
    final_data.extend(i_data)
    i = i+1
df_seeds = pd.DataFrame(final_data)

In [43]:
word_count = {word: 0 for word in all_plant_names}

def count_word_occurrences(names, words):
    for name in names:
        for word in words:
            if word.lower() in name.lower(): 
                word_count[word] += 1
                
                
all_names_df = df_seeds['Название'].tolist()
count_word_occurrences(all_names_df, all_plant_names)

sorted_word_count = sorted(word_count.items(), key=lambda x: x[1], reverse=True)

for word, count in sorted_word_count:
    print(f"Слово '{word}' встречается {count} раз.")

Слово 'Традесканция' встречается 111 раз.
Слово 'Бор' встречается 44 раз.
Слово 'Фикус' встречается 29 раз.
Слово 'Лен' встречается 25 раз.
Слово 'Замиокулькас' встречается 23 раз.
Слово 'Ива' встречается 22 раз.
Слово 'Адениум' встречается 18 раз.
Слово 'Плющ' встречается 18 раз.
Слово 'Томат' встречается 16 раз.
Слово 'Фикус Бенджамина' встречается 14 раз.
Слово 'Эпипремнум' встречается 14 раз.
Слово 'Сансевиерия' встречается 13 раз.
Слово 'Хамедорея' встречается 12 раз.
Слово 'Антуриум' встречается 10 раз.
Слово 'Филодендрон' встречается 10 раз.
Слово 'Ацена' встречается 9 раз.
Слово 'Бегония' встречается 9 раз.
Слово 'Драцена' встречается 9 раз.
Слово 'Колеус' встречается 9 раз.
Слово 'Перец' встречается 9 раз.
Слово 'Рами' встречается 9 раз.
Слово 'Ель' встречается 8 раз.
Слово 'Кротон' встречается 8 раз.
Слово 'Мята' встречается 8 раз.
Слово 'Пеперомия' встречается 8 раз.
Слово 'Примула' встречается 8 раз.
Слово 'Эхинокактус' встречается 8 раз.
Слово 'Крассула' встречается 7 раз.

In [1]:
class Plant:
    def __init__(self, id, species, growth_rate, health_status, age, type, rental_price, area_required, maintenance_cost, original_price, discount_rate):
        self.id = id  
        self.species = species  
        self.growth_rate = growth_rate  #скорость роста (например, см/месяц)
        self.health_status = health_status  #состояние здоровья (good, average, poor)
        self.age = age  
        self.type = type  
        self.rental_price = rental_price  #стоимость аренды в месяц
        self.area_required = area_required  #площадь, которую занимает растение (в м²)
        self.original_price = original_price  #оригинальная цена растения
        self.discount_rate = discount_rate  #скидка на растение (в процентах)
        self.discounted_price = original_price * (1 - discount_rate / 100)  #цена со скидкой
        self.is_rented = False  #флаг, указывающий, арендовано ли растение
        self.is_sold = False  #флаг, указывающий, продано ли растение
        
        
        global plant_id_counter
        plant_id_counter += 1  #обновление id

    def grow(self):
        #учет состояния здоровья: если состояние здоровья плохое, скорость роста снижается, если хорошее, то повышается
        if self.health_status == "poor":
            return max(0, self.growth_rate - 0.2)
        elif self.health_status == "good":
            return max(0, self.growth_rate + 0.2)
        else:
            return self.growth_rate

    def water(self):
        #полив растения улучшает его состояние здоровья
        if self.health_status == "poor":
            self.health_status = "average"
        elif self.health_status == "average":
            self.health_status = "good"
        #если здоровье уже хорошее, полив не меняет его

    def fertilize(self):
        self.growth_rate += 0.2

    def info(self):
        #общая информация о растении
        return (f"Растение {self.id}: вид {self.species}, "
                f"скорость роста: {self.growth_rate} см/месяц, "
                f"здоровье: {self.health_status}, возраст: {self.age} лет, "
                f"тип: {self.type}, стоимость аренды: {self.rental_price} руб/месяц, "
                f"площадь: {self.area_required} м², "
                f"стоимость обслуживания: {self.maintenance_cost} руб/месяц, "
                f"оригинальная цена: {self.original_price} руб, "
                f"скидка: {self.discount_rate}%, цена со скидкой: {self.discounted_price} руб, "
                f"арендовано: {'Да' if self.is_rented else 'Нет'}, "
                f"продано: {'Да' if self.is_sold else 'Нет'}")

In [ ]:
class BusinessModel:
    def __init__(self, warehouse_area, warehouse_rent_cost):
        self.plants = []  
        self.warehouse_area = warehouse_area  
        self.used_area = 0  #занятая площадь на складе (в м²)
        self.total_revenue = 0  #общий доход от аренды
        self.warehouse_rent_cost = warehouse_rent_cost  #стоимость аренды склада в месяц

    def add_plant(self, plant):
        #добавление растения в бизнес-модель с проверкой площади
        if self.used_area + plant.area_required <= self.warehouse_area:
            self.plants.append(plant)
            self.used_area += plant.area_required
            return f"Растение {plant.id} добавлено. Занято площади: {self.used_area} м²."
        else:
            return "Недостаточно места на складе для добавления растения."

    def rent_plant(self, plant_id, duration_months):
        plant = next((p for p in self.plants if p.id == plant_id), None)
        if plant:
            if not plant.is_rented:
                plant.is_rented = True
                revenue = plant.rental_price * duration_months
                self.total_revenue += revenue
                return f"Растение {plant.id} арендовано. Доход: {revenue} руб."
            else:
                return "Растение уже арендовано."
        else:
            return "Растение не найдено."
        
    def sell_plant(self, plant_id):
        #продажа растения по скидке
        plant = next((p for p in self.plants if p.id == plant_id), None)
        if plant:
            if not plant.is_sold and not plant.is_rented:
                plant.is_sold = True
                self.total_revenue += plant.discounted_price
                self.used_area -= plant.area_required  # Освобождаем площадь
                return f"Растение {plant.id} продано по скидке. Доход: {plant.discounted_price} руб."
            else:
                return "Растение уже продано или арендовано."
        else:
            return "Растение не найдено."
        
    def return_plant(self, plant_id):
        #возврат растения
        plant = next((p for p in self.plants if p.id == plant_id), None)
        if plant and plant.is_rented:
            plant.is_rented = False
            return f"Растение {plant.id} возвращено."
        else:
            return "Растение не найдено или не арендовано."

    def calculate_total_revenue(self):
        return self.total_revenue
    
    def calculate_total_expenses(self):
        return self.total_expenses + self.warehouse_rent_cost

    def list_available_plants(self):
        return [plant for plant in self.plants if not plant.is_rented]

    def list_rented_plants(self):
        return [plant for plant in self.plants if plant.is_rented]
    
    def list_sold_plants(self):
        return [plant for plant in self.plants if plant.is_sold]

    def get_warehouse_status(self):
        #статус склада: занятая и свободная площадь
        free_area = self.warehouse_area - self.used_area
        return f"Занято площади: {self.used_area} м², свободно: {free_area} м²"